In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

## Creating subagents

In [3]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [4]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)


## Calling subagents

In [7]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [8]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [9]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='098a5da3-6a20-4a5c-88a7-c2b62f86aea5'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-26T21:29:17.21922Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1907613000, 'load_duration': 85173100, 'prompt_eval_count': 259, 'prompt_eval_duration': 763063700, 'eval_count': 23, 'eval_duration': 921918600, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b5c90-d13f-72a2-8f4e-5935fc7e4bc4-0', tool_calls=[{'name': 'call_subagent_1', 'args': {'x': 456}, 'id': 'efe56fe2-38e0-4c00-b2ac-7135fc113b9a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 259, 'output_tokens': 23, 'total_tokens': 282}),
              ToolMessage(content='The square root of 456 is approximately 21.354.', name='call_subagent_1', id='e683b56a-ca6d-43c6-8989-002044fa3885', tool_